# Step 0. Code Modules, Links and Helpful Functions
#### [📑 GitHub Repository](https://github.com/OlgaBelitskaya/deep_learning_projects/tree/master/DL_PP2) & [📑 Colaboratory](https://colab.research.google.com/drive/1Z9Fz0OOi6bpWvH-H2OhExC9CkGPWBYZz)
#### [📑 Full Version - Python](https://olgabelitskaya.github.io/kaggle_letters.html) & [📑 Full Version - R](https://olgabelitskaya.github.io/kaggle_letters_R.html) 
#### [📑 Deep Learning. P2: Multi-Label Classification. Letter Recognition](https://olgabelitskaya.github.io/DL_PP2_Solutions_SMC.html)

In [ ]:
import warnings; warnings.filterwarnings('ignore')
import numpy as np,pandas as pd,pylab as pl,h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from IPython import display
from keras.preprocessing import image as ksimage
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from keras.metrics import top_k_categorical_accuracy,categorical_accuracy
from keras.models import Sequential,load_model,Model
from keras.layers import Dense,LSTM,GlobalAveragePooling1D,GlobalAveragePooling2D
from keras.layers.advanced_activations import PReLU,LeakyReLU
from keras.layers import Input,Activation,Flatten,Dropout,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D,GlobalMaxPooling2D
fw='weights.best.letters.hdf5'

In [ ]:
# plotting of fitting histories for neural networks
def history_plot2(fit_history):
    keys=list(fit_history.history.keys())[6:]
    pl.figure(figsize=(12,10)); pl.subplot(211)
    pl.plot(fit_history.history[keys[0]],
            color='slategray',label='valid 1')
    pl.plot(fit_history.history[keys[1]],
            color='#4876ff',label='valid 2')
    pl.xlabel("Epochs"); pl.ylabel("Loss")
    pl.legend(); pl.grid(); pl.title('Loss Function')     
    pl.subplot(212)
    pl.plot(fit_history.history[keys[2]],
            color='slategray',label='valid 1')
    pl.plot(fit_history.history[keys[3]],
            color='#4876ff',label='valid 2')
    pl.xlabel("Epochs"); pl.ylabel("Accuracy")    
    pl.legend(); pl.grid(); pl.title('Accuracy'); pl.show()
# preprocessing functions 
def ohe(x): 
    return OneHotEncoder(categories='auto')\
           .fit(x.reshape(-1,1)).transform(x.reshape(-1,1))\
           .toarray().astype('int64')
def tts(X,y): 
    x_train,x_test,y_train,y_test=\
    train_test_split(X,y,test_size=.2,random_state=1)
    n=int(len(x_test)/2)
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    return x_train,x_valid,x_test,y_train,y_valid,y_test

# ✒️ Step 1. Loading and Preprocessing the Data

In [ ]:
f=h5py.File('../input/LetterColorImages_123.h5','r')
keys=list(f.keys()); keys

In [ ]:
# creating image arrays and targets
letters=u'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
backgrounds=np.array(f[keys[0]])
labels=np.array(f[keys[2]])
# normalization of image arrays
images=np.array(f[keys[1]])/255

In [ ]:
pl.figure(figsize=(2,3)); il=10**4
pl.title('Label: %s \n'%letters[labels[il]-1]+\
         'Background: %s'%backgrounds[il],
         fontsize=18)
pl.imshow(images[il]); pl.show()

In [ ]:
# one-hot encoding
cbackgrounds,clabels=ohe(backgrounds),ohe(labels)
ctargets=np.concatenate((clabels,cbackgrounds),axis=1)
display.display(pd.DataFrame([labels[97:103],clabels[97:103]]).T)
pd.DataFrame([clabels.shape,cbackgrounds.shape,ctargets.shape])

In [ ]:
# splitting the data
x_train,x_valid,x_test,\
y_train,y_valid,y_test=tts(images,ctargets)

In [ ]:
y_train_list=[y_train[:,:33],y_train[:,33:]]
y_test_list=[y_test[:,:33],y_test[:,33:]]
y_valid_list=[y_valid[:,:33],y_valid[:,33:]]

# ✒️ Step 2. Defining a Multi-Label Classification Model

In [ ]:
def multi_model():   
    model_input=Input(shape=(32,32,3))
    x=BatchNormalization()(model_input)    
    x=Conv2D(32,(5,5),padding='same')(model_input)
    x=LeakyReLU(alpha=.02)(x)
    x=MaxPooling2D(pool_size=(2,2))(x)    
    x=Dropout(.25)(x)    
    x=Conv2D(256,(5,5),padding='same')(x)  
    x=LeakyReLU(alpha=.02)(x)
    x=MaxPooling2D(pool_size=(2,2))(x)    
    x=Dropout(.25)(x)              
    x=GlobalMaxPooling2D()(x)   
    x=Dense(1024)(x) 
    x=LeakyReLU(alpha=.02)(x)
    x=Dropout(.25)(x)       
    y1=Dense(33,activation='softmax')(x)
    y2=Dense(4,activation='softmax')(x)    
    model=Model(inputs=model_input,outputs=[y1,y2])
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',metrics=['accuracy'])
    return model
multi_model=multi_model()

In [ ]:
multi_model.summary()

In [ ]:
checkpointer=ModelCheckpoint(filepath=fw,verbose=2,save_best_only=True)
lr_reduction=ReduceLROnPlateau(monitor='val_loss',patience=5,verbose=2,factor=.75)
estopping=EarlyStopping(monitor='val_loss',patience=20,verbose=2)
history=multi_model.fit(x_train,y_train_list,
                        epochs=200,batch_size=64,verbose=2,
                        validation_data=(x_valid,y_valid_list),
                        callbacks=[checkpointer,lr_reduction,estopping])

In [ ]:
history_plot2(history)
# loading the model weights with the best validation accuracy
multi_model.load_weights(fw)
multi_model.evaluate(x_test,y_test_list)